<a href="https://colab.research.google.com/github/nvvishnu/CS6910-Assignments/blob/main/Assignment2/partB/Assignment2_partB_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras import *
!pip install wandb
import wandb
from wandb.keras import WandbCallback 
import math

     |████████████████████████████████| 2.1MB 4.4MB/s 
     |████████████████████████████████| 163kB 18.3MB/s 
     |████████████████████████████████| 102kB 5.7MB/s 
     |████████████████████████████████| 133kB 18.3MB/s 
     |████████████████████████████████| 71kB 6.3MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=ffb9aec39c7e9be6be25b56fdba7cdf6c04b9e368a099dcd8a2b76ff7ab44f86
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=9968f0f87392c96a24b17261fda44941c52829bb6ce2e3e1e7abb3d8001ea3a5
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools


In [2]:
def train_model(x,retrain_top_layers_percentage,augmentation,epochs,target_size,optimizer):
    
    g = len(x.layers)
    print('layers_size: ',g)
    num_retrain = math.ceil(g*(1-retrain_top_layers_percentage))
    for gg in x.layers[:num_retrain]:
      gg.trainable = False
    print(num_retrain)
    model = Sequential()
    model.add(x)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Flatten())
    model.add(layers.Dense(1024,activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(10))
    model.add(layers.Softmax())
    model.summary()
    datagen = preprocessing.image.ImageDataGenerator()
    path = '/content/drive/MyDrive/Assignment2/inaturalist_12K/'
    augmented = preprocessing.image.ImageDataGenerator(
                              #featurewise_center=True,
                              #featurewise_std_normalization=True,
                              rotation_range=20,
                              width_shift_range=0.2,
                              height_shift_range=0.2,
                              horizontal_flip=True)
    
    if(augmentation):
      train = augmented.flow_from_directory(path+'split/train/',target_size=target_size,color_mode='rgb', classes=None,class_mode='categorical', batch_size=32)
    else:
       train = datagen.flow_from_directory(path+'split/train/',target_size=target_size,color_mode='rgb', classes=None,class_mode='categorical', batch_size=32)
    validation = datagen.flow_from_directory(path+'split/val/',target_size=target_size,color_mode='rgb', classes=None,class_mode='categorical', batch_size=32)
     #x is input model
    model.compile(optimizer=optimizer,loss=losses.CategoricalCrossentropy(),metrics=['accuracy'])
    model.fit(train,epochs=epochs,validation_data=validation) #,callbacks=[WandbCallback()])

    return model

def fun(model_name,retrain_top_layers_percentage,augmentation,epochs,optimizer):
    
    if(model_name == 'NASNetLarge'):
       x = applications.NASNetLarge(include_top=False, weights='imagenet',input_shape=(331,331,3))
       target_size = (331,331)
    elif(model_name == 'InceptionV3'):
       x = applications.InceptionV3(include_top=False, weights='imagenet',input_shape=(224,224,3))
       target_size = (224,224)
    elif(model_name == 'ResNet50'):
       x = applications.ResNet50(include_top=False, weights='imagenet',input_shape=(224,224,3))
       target_size = (224,224)
    elif(model_name == 'Xception'):
       x = applications.Xception(include_top=False, weights='imagenet',input_shape=(224,224,3))
       target_size = (224,224)
    elif(model_name == 'InceptionResNetV2'):
       x = applications.InceptionResNetV2(include_top=False, weights='imagenet',input_shape=(224,224,3))
       target_size = (224,224)
    elif(model_name == 'ResNet101'):
      x = applications.ResNet101(include_top=False,weights='imagenet',input_shape=(224,224,3))
      target_size = (224,224)
    elif(model_name == 'ResNet50V2'):
      x = applications.ResNet50V2(include_top=False,weights='imagenet',input_shape=(224,224,3))
      target_size = (224,224)
    elif(model_name == 'VGG16'):
      x = applications.VGG16(include_top=False,weights='imagenet',input_shape=(224,224,3))
      target_size = (224,224)
    else:
      raise Exception("Invalid Model name")
    
    return train_model(x,retrain_top_layers_percentage,augmentation,epochs,target_size,optimizer)
    

In [3]:
sweep_config = {
    'name': 'tuning_model',
    'method' : 'bayes',
    'metric': {        
         'goal': 'maximize',
         'name' : 'val_accuracy'
    },
    'parameters': {
        'model_name' : {
            'values' : ['NASNetLarge','ResNet101','ResNet50','ResNet50V2','VGG16','InceptionV3','InceptionResNetV2']
        },
        'epochs': {
            'values':[5,6]
        },
        'retrain_top_layers_percentage':{
            'values':[0.1,0.2,0.3]
        },
        'optimizer':{
            'values':['adam','sgd']
        }
    }
 }

In [4]:
def train():
  config_default = { 
      'model_name' : 'ResNet50',
      'epochs': 10,
      'retrain_top_layers_percentage': 0.2,
      'augmentation':True,
      'optimizer':'adam'
  }
  print("wand about to be initialised")
  wandb.init(config=config_default,project='Assignment2')
  config = wandb.config

  fun(config.model_name,config.retrain_top_layers_percentage,True,config.epochs,config.optimizer)

In [5]:
#sweep_id = wandb.sweep(sweep_config,entity="shivaram_22",project="Assignment2")
#wandb.agent(sweep_id,project="Assignment2",function=train)

In [8]:
#this our best model
best_model = fun('ResNet50',0.2,True,5,'sgd')

layers_size:  175
140
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                10250     
_________________________________________________________________
softmax_1 (Softmax)          (No

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
